In [1]:
!pip install https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/en_legal_ner_trf-any-py3-none-any.whl
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446.7 MB 23 kB/s 
     |████████████████████████████████| 6.2 MB 10.0 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 13.7 MB 50.3 MB/s 
     |████████████████████████████████| 671 kB 70.9 MB/s 
     |████████████████████████████████| 4.7 MB 57.7 MB/s 
     |████████████████████████████████| 53 kB 2.6 MB/s 
     |████████████████████████████████| 4.4 MB 61.7 MB/s 
     |████████████████████████████████| 1.1 MB 71.4 MB/s 
     |████████████████████████████████| 6.6 MB 42.0 MB/s 
     |████████████████████████████████| 182 kB 84.3 MB/s 
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
  

In [ ]:
cd '/content/drive/MyDrive/NLP/NLP project/legal_NER-main'

In [ ]:
!pip install -r requirements.txt

In [2]:
import pandas as pd

with open('/content/NER_TRAIN_PREAMBLE.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

df.to_csv('csvfile.csv', encoding='utf-8', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df

,id,annotations,data,meta
0,d79fb7f965a74e418212458285c7c213,"[{'result': [{'value': {'start': 7, 'end': 40,...",{'text': 'In The High Court Of Kerala At Ernak...,{'source': 'criminal_kerala_high_court preambl...
1,10d82191b30e4e9d9b0ab91c34606631,"[{'result': [{'value': {'start': 21, 'end': 40...",{'text': 'In The Court Of Shri Lokesh Kumar Sh...,{'source': 'constitution_districtcourts preamb...
2,b150ace41ceb4acf87d12f75b4748ffc,"[{'result': [{'value': {'start': 11, 'end': 45...",{'text': 'Before The Madurai Bench Of Madras H...,{'source': 'civil_madras_high_court preamble h...
3,278f93b03a484705b3d96a72aa09be26,"[{'result': [{'value': {'start': 11, 'end': 45...",{'text': 'Before The Madurai Bench Of Madras H...,{'source': 'tax_madras_high_court preamble htt...
4,ceab6a1b200242208a4d6beb042d8743,"[{'result': [{'value': {'start': 85, 'end': 11...",{'text': '1 ...,{'source': 'criminal_bombay_high_court preambl...
...,...,...,...,...
1555,2b617380eac44e85a80d53e9c067360b,"[{'result': [{'value': {'start': 7, 'end': 41,...",{'text': 'In The High Court Of Judicature At M...,{'source': 'motorvehicles_madras_high_court pr...
1556,799133f87be04599a0b5367a4a1a23bf,"[{'result': [{'value': {'start': 12, 'end': 24...",{'text': 'Petitioner: Khudiram Das Vs. Resp...,{'source': 'criminal_supremecourts preamble ht...
1557,d21055fa778e4579a5711c6ffb1b9560,"[{'result': [{'value': {'start': 7, 'end': 40,...",{'text': 'In The High Court Of Judicature At P...,{'source': 'constitution_patna_high_court prea...
1558,307722b0639841c9b31418fbc84d7a52,"[{'result': [{'value': {'start': 16, 'end': 52...",{'text': 'Petitioner: The Automobile Transport...,{'source': 'tax_supremecourts preamble https:/...


In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Text Preprocessing

### Preproceesing annotations start

In [5]:
annotations_column = df['annotations']

In [6]:
annotations_column =  annotations_column.to_numpy()

In [7]:
annotation_start = []
annotation_end = []


for sent in annotations_column:
  sent_start = []
  sent_end = []
  sent_text = []
  sent_label = []
  for sent_res in sent:
    for res_value in sent_res['result']:
      sent_start.append(res_value['value']['start'])

    for res_value in sent_res['result']:
      sent_end.append(res_value['value']['end'])
    


  annotation_start.append(sent_start)
  annotation_end.append(sent_end)


In [8]:
annotation_start[0]

[7, 73, 171, 237, 304, 350, 391, 425]

In [9]:
len(annotation_end)

1560

In [10]:
annotation_text = []
annotation_label = []
for sent in annotations_column:
    sent_text = []
    sent_label = []
    for sent_res in sent:

        for res_value in sent_res['result']:
          sent_text.append(res_value['value']['text'])
        
        for res_value in sent_res['result']:
          sent_label.append(res_value['value']['labels']) 

    annotation_text.append(sent_text)
    annotation_label.append(sent_label)

In [11]:
annotation_text[0]

['High Court Of Kerala At Ernakulam',
 'T.R.Ajayan',
 'M.Ravindran',
 'Nirmala Dinesh',
 'A.Kumar',
 'M.K.Pushpalatha',
 'P.R.Raman',
 'V.K.Mohanan']

In [12]:
annotation_label[0]

[['COURT'],
 ['PETITIONER'],
 ['RESPONDENT'],
 ['RESPONDENT'],
 ['LAWYER'],
 ['LAWYER'],
 ['JUDGE'],
 ['JUDGE']]

In [13]:
column_names = ['start', 'end' , 'text' , 'labels']

final_df = pd.DataFrame()
final_df['start'] = annotation_start
final_df['end'] = annotation_end
final_df['text'] = annotation_text
final_df['labels'] = annotation_label

final_df.head()

,start,end,text,labels
0,"[7, 73, 171, 237, 304, 350, 391, 425]","[40, 83, 182, 251, 311, 365, 400, 436]","[High Court Of Kerala At Ernakulam, T.R.Ajayan...","[[COURT], [PETITIONER], [RESPONDENT], [RESPOND..."
1,"[21, 371, 445, 689, 741, 852, 1007, 1035, 1111...","[40, 383, 470, 698, 764, 869, 1031, 1047, 1121...","[Lokesh Kumar Sharma, Chanakyapuri, Jindal Ste...","[[JUDGE], [PETITIONER], [RESPONDENT], [RESPOND..."
2,"[11, 98, 192, 208, 219, 237, 253, 265, 281, 29...","[45, 115, 204, 215, 233, 249, 261, 277, 295, 3...","[Madurai Bench Of Madras High Court, V.Ramasub...","[[COURT], [JUDGE], [PETITIONER], [PETITIONER],..."
3,"[11, 99, 144, 305, 461, 593, 695, 1335, 1364, ...","[45, 112, 229, 330, 473, 604, 784, 1350, 1375,...","[Madurai Bench Of Madras High Court, T.Sudanth...","[[COURT], [JUDGE], [PETITIONER], [RESPONDENT],..."
4,"[85, 435, 614, 773, 879, 997, 1276, 1598, 1658...","[119, 460, 637, 798, 887, 1035, 1296, 1617, 16...","[High Court Of Judicature At Bombay, Motiram M...","[[COURT], [PETITIONER], [RESPONDENT], [PETITIO..."


In [14]:
for a,b,c,d in zip(annotation_start[:2], annotation_end[:2],annotation_text[:2],annotation_label[:2]):
  print(a,b,c,d)

[7, 73, 171, 237, 304, 350, 391, 425] [40, 83, 182, 251, 311, 365, 400, 436] ['High Court Of Kerala At Ernakulam', 'T.R.Ajayan', 'M.Ravindran', 'Nirmala Dinesh', 'A.Kumar', 'M.K.Pushpalatha', 'P.R.Raman', 'V.K.Mohanan'] [['COURT'], ['PETITIONER'], ['RESPONDENT'], ['RESPONDENT'], ['LAWYER'], ['LAWYER'], ['JUDGE'], ['JUDGE']]
[21, 371, 445, 689, 741, 852, 1007, 1035, 1111, 1245] [40, 383, 470, 698, 764, 869, 1031, 1047, 1121, 1255] ['Lokesh Kumar Sharma', 'Chanakyapuri', 'Jindal Steel & Power Ltd.', 'The State', 'Planman Media Pvt. Ltd.', 'Arindam Chaudhuri', 'Jindal Steel & Power Ltd', 'State & Ors.', 'A. Sandeep', 'Ashok Bose'] [['JUDGE'], ['PETITIONER'], ['RESPONDENT'], ['RESPONDENT'], ['RESPONDENT'], ['RESPONDENT'], ['RESPONDENT'], ['RESPONDENT'], ['LAWYER'], ['LAWYER']]


In [15]:
zip(annotation_start[0], annotation_end[0])

In [16]:
pwd

'/content'

In [17]:
final_df.to_csv('Processed_DataFrame_Train')

## IOB tagging

In [18]:
import pandas as pd

df_ = pd.read_csv('/content/Processed_DataFrame_Train')
df_.head()

,Unnamed: 0,start,end,text,labels
0,0,"[7, 73, 171, 237, 304, 350, 391, 425]","[40, 83, 182, 251, 311, 365, 400, 436]","['High Court Of Kerala At Ernakulam', 'T.R.Aja...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['..."
1,1,"[21, 371, 445, 689, 741, 852, 1007, 1035, 1111...","[40, 383, 470, 698, 764, 869, 1031, 1047, 1121...","['Lokesh Kumar Sharma', 'Chanakyapuri', 'Jinda...","[['JUDGE'], ['PETITIONER'], ['RESPONDENT'], ['..."
2,2,"[11, 98, 192, 208, 219, 237, 253, 265, 281, 29...","[45, 115, 204, 215, 233, 249, 261, 277, 295, 3...","['Madurai Bench Of Madras High Court', 'V.Rama...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['PETIT..."
3,3,"[11, 99, 144, 305, 461, 593, 695, 1335, 1364, ...","[45, 112, 229, 330, 473, 604, 784, 1350, 1375,...","['Madurai Bench Of Madras High Court', 'T.Suda...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['RESPO..."
4,4,"[85, 435, 614, 773, 879, 997, 1276, 1598, 1658...","[119, 460, 637, 798, 887, 1035, 1296, 1617, 16...","['High Court Of Judicature At Bombay', 'Motira...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['..."


In [19]:
text = df.iloc[[0]]['data']

In [20]:
text[0]['text']

"In The High Court Of Kerala At Ernakulam\n\nCrl Mc No. 1622 of 2006()\n\n\n1. T.R.Ajayan, S/O. O.Raman,\n                      ...  Petitioner\n\n                        Vs\n\n\n\n1. M.Ravindran,\n                       ...       Respondent\n\n2. Mrs. Nirmala Dinesh, W/O. Dinesh,\n\n                For Petitioner  :Sri.A.Kumar\n\n                For Respondent  :Smt.M.K.Pushpalatha\n\nThe Hon'ble Mr. Justice P.R.Raman\nThe Hon'ble Mr. Justice V.K.Mohanan\n\n Dated :07/01/2008\n\n O R D E R\n"

In [21]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(text[0]['text'])

for d in doc:
  print(d.text)

In
The
High
Court
Of
Kerala
At
Ernakulam



Crl
Mc
No
.
1622
of
2006
(
)




1
.
T.R.Ajayan
,
S
/
O.
O.Raman
,

                      
...
 
Petitioner


                        
Vs





1
.
M.Ravindran
,

                       
...
      
Respondent



2
.
Mrs.
Nirmala
Dinesh
,
W
/
O.
Dinesh
,


                
For
Petitioner
 
:
Sri
.
A.Kumar


                
For
Respondent
 
:
Smt
.
M.K.Pushpalatha



The
Hon'ble
Mr.
Justice
P.R.Raman


The
Hon'ble
Mr.
Justice
V.K.Mohanan


 
Dated
:
07/01/2008


 
O
R
D
E
R




In [22]:
import re
t = re.sub('\n', " ", text[0]['text'])


In [23]:
" ".join(t.split())

"In The High Court Of Kerala At Ernakulam Crl Mc No. 1622 of 2006() 1. T.R.Ajayan, S/O. O.Raman, ... Petitioner Vs 1. M.Ravindran, ... Respondent 2. Mrs. Nirmala Dinesh, W/O. Dinesh, For Petitioner :Sri.A.Kumar For Respondent :Smt.M.K.Pushpalatha The Hon'ble Mr. Justice P.R.Raman The Hon'ble Mr. Justice V.K.Mohanan Dated :07/01/2008 O R D E R"

In [24]:
df['data']

0       {'text': 'In The High Court Of Kerala At Ernak...
1       {'text': 'In The Court Of Shri Lokesh Kumar Sh...
2       {'text': 'Before The Madurai Bench Of Madras H...
3       {'text': 'Before The Madurai Bench Of Madras H...
4       {'text': '1                                   ...
                              ...                        
1555    {'text': 'In The High Court Of Judicature At M...
1556    {'text': 'Petitioner:
Khudiram Das

	Vs.

Resp...
1557    {'text': 'In The High Court Of Judicature At P...
1558    {'text': 'Petitioner:
The Automobile Transport...
1559    {'text': 'Petitioner:
East India COMMERClAL Co...
Name: data, Length: 1560, dtype: object

In [25]:
def preprocess(sent):
  sent = re.sub('\n', " ", sent)
  sent = sent.replace("...", " ")
  sent = " ".join(sent.split())

  return sent


clean_text = []
for i in df['data']:
  clean_text.append(preprocess(i['text']))

In [26]:
len(clean_text)

1560

In [27]:
df_['clean_text'] = clean_text

In [28]:
df_.head()

,Unnamed: 0,start,end,text,labels,clean_text
0,0,"[7, 73, 171, 237, 304, 350, 391, 425]","[40, 83, 182, 251, 311, 365, 400, 436]","['High Court Of Kerala At Ernakulam', 'T.R.Aja...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['...",In The High Court Of Kerala At Ernakulam Crl M...
1,1,"[21, 371, 445, 689, 741, 852, 1007, 1035, 1111...","[40, 383, 470, 698, 764, 869, 1031, 1047, 1121...","['Lokesh Kumar Sharma', 'Chanakyapuri', 'Jinda...","[['JUDGE'], ['PETITIONER'], ['RESPONDENT'], ['...",In The Court Of Shri Lokesh Kumar Sharma Addit...
2,2,"[11, 98, 192, 208, 219, 237, 253, 265, 281, 29...","[45, 115, 204, 215, 233, 249, 261, 277, 295, 3...","['Madurai Bench Of Madras High Court', 'V.Rama...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['PETIT...",Before The Madurai Bench Of Madras High Court ...
3,3,"[11, 99, 144, 305, 461, 593, 695, 1335, 1364, ...","[45, 112, 229, 330, 473, 604, 784, 1350, 1375,...","['Madurai Bench Of Madras High Court', 'T.Suda...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['RESPO...",Before The Madurai Bench Of Madras High Court ...
4,4,"[85, 435, 614, 773, 879, 997, 1276, 1598, 1658...","[119, 460, 637, 798, 887, 1035, 1296, 1617, 16...","['High Court Of Judicature At Bombay', 'Motira...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['...","1 In The High Court Of Judicature At Bombay, N..."


In [29]:
s = df_.iloc[0]['clean_text']

for i in range(7,40):
  print(s[i])

H
i
g
h
 
C
o
u
r
t
 
O
f
 
K
e
r
a
l
a
 
A
t
 
E
r
n
a
k
u
l
a
m


In [30]:
df_.iloc[1]['clean_text']

import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(df_.iloc[0]['clean_text'])

print(doc[0])

In


In [31]:
span = doc.char_span(73,100,label = "COURT")
span

## New solution

In [32]:
def sentence_cleaner(sentence):
    '''
    replacing the unwanted charachters
    '''
    reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
    return " ".join(reg.findall(sentence.replace('\n', ' '))).replace(" / ", '/')

In [33]:
def sentence_cleaner_v1(sentence):
    '''
    replacing the unwanted charachters
    '''
    sent_1 = " ".join(sentence.replace("\n" , " ").replace("..." , " ").replace("-", " ").replace("," , " ").replace(":" , " ").replace("=",  " ").split())
    doc = nlp(sent_1)
    doc_list = []
    for i in doc:
      doc_list.append(str(i))
    return doc_list

In [34]:
def sentence_tagging(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,temp_len)
    if temp_len > 1 :
        for i in range(temp_len):
            #print(i, temp_len - 1)
            if i == 0 :
              word2ent[temp_sent[i]] =  "B-" + ann['value']['labels'][0]
            
            elif i == (temp_len - 1):
              word2ent[temp_sent[i]] =  "E-" + ann['value']['labels'][0]

            else :
              word2ent[temp_sent[i]] =  "I-" + ann['value']['labels'][0]

    else :
        word2ent[temp_sent[0]] = "B-" + ann['value']['labels'][0]

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys() and word + ',' not in word2ent.keys() :
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
        tag_list.append(word2ent[word])
      
      else :
          tag_list.append(word2ent[word])
  
  
  print(word2ent )
  return tag_list
      

In [35]:
def sentence_tagging_v1(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
      tag_list.append(word2ent[word])
      
  
  
  #print(word2ent )
  return tag_list
      

In [36]:
def sentence_tagging_v2(sent, annotation):

  """
  current using word
  """

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = sentence_cleaner_v1(sent[ ann['value']['start']: ann['value']['end']])
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner_v1(sent)
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        tag = "O"
      else :
        if len(word2ent[word]) != 0 :
          tag = word2ent[word].pop(0)
        else :
          tag = "O"

      #print(word,tag)   
      tag_list.append(str(tag))
      
  
  
  #print(word2ent )
  return tag_list
      

In [37]:
IOBE_tagging = []
i = 0
for sent, anno in zip( df['data'].to_list(), df['annotations'].to_list()):
    IOBE_tagging.append(sentence_tagging_v2(sent,anno))
    
  

In [38]:
IOBE_tagging[2]

['O',
 'O',
 'B-COURT',
 'I-COURT',
 'I-COURT',
 'I-COURT',
 'I-COURT',
 'E-COURT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-JUDGE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-PETITIONER',
 'B-PETITIONER',
 'E-PETITIONER',
 'O',
 'O',
 'B-PETITIONER',
 'O',
 'O',
 'B-PETITIONER',
 'E-PETITIONER',
 'O',
 'O',
 'B-PETITIONER',
 'E-PETITIONER',
 'O',
 'O',
 'B-PETITIONER',
 'E-PETITIONER',
 'O',
 'O',
 'B-PETITIONER',
 'E-PETITIONER',
 'O',
 'O',
 'B-PETITIONER',
 'E-PETITIONER',
 'O',
 'O',
 'B-PETITIONER',
 'E-PETITIONER',
 'O',
 'O',
 'B-PETITIONER',
 'E-PETITIONER',
 'B-PETITIONER',
 'E-PETITIONER',
 'B-PETITIONER',
 'E-PETITIONER',
 'B-PETITIONER',
 'E-PETITIONER',
 'B-PETITIONER',
 'B-PETITIONER',
 'O',
 'E-PETITIONER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-RESPONDENT',
 'E-RESPONDENT',
 'O',
 'O',
 'B-RESPONDENT',
 'O',
 'O',
 'B-RESPONDENT',
 'E-RESPONDENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',


In [39]:
final_df['tags'] = IOBE_tagging
final_df.head()

,start,end,text,labels,tags
0,"[7, 73, 171, 237, 304, 350, 391, 425]","[40, 83, 182, 251, 311, 365, 400, 436]","[High Court Of Kerala At Ernakulam, T.R.Ajayan...","[[COURT], [PETITIONER], [RESPONDENT], [RESPOND...","[O, O, B-COURT, I-COURT, I-COURT, I-COURT, I-C..."
1,"[21, 371, 445, 689, 741, 852, 1007, 1035, 1111...","[40, 383, 470, 698, 764, 869, 1031, 1047, 1121...","[Lokesh Kumar Sharma, Chanakyapuri, Jindal Ste...","[[JUDGE], [PETITIONER], [RESPONDENT], [RESPOND...","[O, B-RESPONDENT, O, O, O, B-JUDGE, I-JUDGE, E..."
2,"[11, 98, 192, 208, 219, 237, 253, 265, 281, 29...","[45, 115, 204, 215, 233, 249, 261, 277, 295, 3...","[Madurai Bench Of Madras High Court, V.Ramasub...","[[COURT], [JUDGE], [PETITIONER], [PETITIONER],...","[O, O, B-COURT, I-COURT, I-COURT, I-COURT, I-C..."
3,"[11, 99, 144, 305, 461, 593, 695, 1335, 1364, ...","[45, 112, 229, 330, 473, 604, 784, 1350, 1375,...","[Madurai Bench Of Madras High Court, T.Sudanth...","[[COURT], [JUDGE], [PETITIONER], [RESPONDENT],...","[O, O, B-COURT, I-COURT, I-COURT, I-COURT, I-C..."
4,"[85, 435, 614, 773, 879, 997, 1276, 1598, 1658...","[119, 460, 637, 798, 887, 1035, 1296, 1617, 16...","[High Court Of Judicature At Bombay, Motiram M...","[[COURT], [PETITIONER], [RESPONDENT], [PETITIO...","[O, O, O, B-COURT, I-COURT, I-COURT, I-COURT, ..."


In [40]:
sent_1 = df['data'].to_list()[2]['text']
sent_1

'Before The Madurai Bench Of Madras High Court\n\nDated: 23/12/2011\n\nCoram\nThe Honourable Mr.Justice V.Ramasubramanian\n\nCivil Revision Petition (Npd)(Md) No.1123 of 2006\nAnd\nM.P.No.2 of 2006\n\n1. Ayisha Beevi\n2. Beevija\n3. Hadijath Beevi\n4. Yunusa Begam\n5. Syed Ali\n6. Sumaya Begam\n7. Mohamed Yoosuf\n8. Mohamed Ismail\n9. Razira Beevi\n10.Shabi Mohamed\n11.Zakir Mugain\n12.Ferosh Khan\n13.Augustin\n14.Dr.T.C.Joseph\t\t\t..... Petitioners\n\nVs.\n\n1. Sheik Mydeen\n2. A.P.Nelson\n3. Chandrakala Ruben\t\t\t..... Respondents\n-----\n\tPetition under Article 227 of  the Constitution of India against the fair\nand decretal order dated 18.10.2006 made in E.P.No.201 of 2000 in O.S.No.420 of\n1976 passed by the Principal District Munsif, Padmanabhapuram.\n\n!For Petitioner \t    \t ...   Mr.K.N.Thampi\n^For Respondent-1 \t  \t ...   Mr.Prabhu Rajadurai\nFor Respondents 2&3 \t\t ...   Mr.A.Arumugham\n-----\n:Order'

In [ ]:
# tag_2 = final_df['tags'].to_list()[77]
# tag_2

In [41]:
# [\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]

reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
" ".join(reg.findall(sent_1.replace('\n', ' '))).replace(" / ", '/')

'Before The Madurai Bench Of Madras High Court Dated 23/12/2011 Coram The Honourable Mr.Justice V.Ramasubramanian Civil Revision Petition Npd Md No.1123 of 2006 And M.P.No 2 of 2006 1 Ayisha Beevi 2 Beevija 3 Hadijath Beevi 4 Yunusa Begam 5 Syed Ali 6 Sumaya Begam 7 Mohamed Yoosuf 8 Mohamed Ismail 9 Razira Beevi 10.Shabi Mohamed 11.Zakir Mugain 12.Ferosh Khan 13.Augustin 14.Dr.T.C.Joseph Petitioners Vs 1 Sheik Mydeen 2 A.P.Nelson 3 Chandrakala Ruben Respondents Petition under Article 227 of the Constitution of India against the fair and decretal order dated 18.10.2006 made in E.P.No.201 of 2000 in O.S.No.420 of 1976 passed by the Principal District Munsif Padmanabhapuram For Petitioner Mr.K.N.Thampi For Respondent 1 Mr.Prabhu Rajadurai For Respondents 2 3 Mr.A.Arumugham Order'

In [42]:
" ".join(sent_1.replace(" "," ").split())

'Before The Madurai Bench Of Madras High Court Dated: 23/12/2011 Coram The Honourable Mr.Justice V.Ramasubramanian Civil Revision Petition (Npd)(Md) No.1123 of 2006 And M.P.No.2 of 2006 1. Ayisha Beevi 2. Beevija 3. Hadijath Beevi 4. Yunusa Begam 5. Syed Ali 6. Sumaya Begam 7. Mohamed Yoosuf 8. Mohamed Ismail 9. Razira Beevi 10.Shabi Mohamed 11.Zakir Mugain 12.Ferosh Khan 13.Augustin 14.Dr.T.C.Joseph ..... Petitioners Vs. 1. Sheik Mydeen 2. A.P.Nelson 3. Chandrakala Ruben ..... Respondents ----- Petition under Article 227 of the Constitution of India against the fair and decretal order dated 18.10.2006 made in E.P.No.201 of 2000 in O.S.No.420 of 1976 passed by the Principal District Munsif, Padmanabhapuram. !For Petitioner ... Mr.K.N.Thampi ^For Respondent-1 ... Mr.Prabhu Rajadurai For Respondents 2&3 ... Mr.A.Arumugham ----- :Order'

In [43]:
word2count = {}

for sent in df['data'].to_list():
    for word in sent['text'].split():
        if word not in word2count:
          word2count[word] = 0
        else:
          word2count[word] += 1

In [44]:
for key in word2count.keys():
  if re.search(r'\bM+\b', key):
    print(key)

M.Ravindran,
:Smt.M.K.Pushpalatha
M/s
M.P.No.2
1.M/s.Sree
M/s.Sree
Crl.M.P.No.1472
M.Sasidharan
M.
M.A.C.M.A
M.A.C.M.A.No.1656
M.C.Hari
M.S.Ramachandra
Ms.M.B.Dominique
M/s.
M.M.
M.A.
M.P
M.S.Rajendraprasad,
M.A.C.M.A.
Cm(M)
Smt.M.Saroja,
M
(Spl.),L.L.M.,
M.V.C.No.7875/2012
G.M.S.
A.M.
A.M.D.
Smitha.M.,
Sulekha.M.,
(M.P.)
I.M.
M.S.K.
A.T.M.
M.A.C.T.,
M.V.C.
A.M.Narendra
A.M.Muninagappa,
M.E.Mohan
M.Gopal
M.Venugopal
C.M.A.No.1347
M/s.United
M.C.O.P.No.237
F.M.Ibrahim
Mr.M.Palani
Mr.M.Ajmalkhan
M/S.
K.M.
P.M.C.
A.M.Saraogi
M.P.No.1
M.Seetharama
3.M.Loganathan
M/s.Sudha
M.A.C.M.A.Nos.2967
V.M.Banakar,
:Sri.K.M.Sathyanatha
Crl.M.C.
M.K.
M.Trivedi
Mr.S.M.Mehta,
C.M.A.No.2539
M.A.C.T.O.P.No.747
Mrs.M.Malar
M.C.
M.A.C.A.
Mr.N.M.Lodha,
M.M.Vijayaraghavan,
C.M.A.No.2484
M.C.O.P.No.49
Sh.M
M.P.Nos.1
M/s.Digivision
Mr.M.Sukumar
M/s.Narmada
M/s.N.V.S.Associates
M/s.Murthy
M/s.Rangarajan
M.C.Chacko,
M.A.C.A
M.Chockalingam
K.M.V.Anand
M/s.Mahalaxmi
Mr.M.M.Vaidya
L.L.M.,
M.V.C.No.7561/2012
M.S.
M.F.

## Using Tokenizer

In [45]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

In [46]:
sent_1 = " ".join(sent_1.replace("\n" , " ").replace("..." , " ").replace("-", "").split())
doc = nlp(sent_1)
doc_list = []
for i in doc:
  doc_list.append(str(i))
doc_list

['Before',
 'The',
 'Madurai',
 'Bench',
 'Of',
 'Madras',
 'High',
 'Court',
 'Dated',
 ':',
 '23/12/2011',
 'Coram',
 'The',
 'Honourable',
 'Mr.',
 'Justice',
 'V.Ramasubramanian',
 'Civil',
 'Revision',
 'Petition',
 '(',
 'Npd)(Md',
 ')',
 'No.1123',
 'of',
 '2006',
 'And',
 'M.P.No.2',
 'of',
 '2006',
 '1',
 '.',
 'Ayisha',
 'Beevi',
 '2',
 '.',
 'Beevija',
 '3',
 '.',
 'Hadijath',
 'Beevi',
 '4',
 '.',
 'Yunusa',
 'Begam',
 '5',
 '.',
 'Syed',
 'Ali',
 '6',
 '.',
 'Sumaya',
 'Begam',
 '7',
 '.',
 'Mohamed',
 'Yoosuf',
 '8',
 '.',
 'Mohamed',
 'Ismail',
 '9',
 '.',
 'Razira',
 'Beevi',
 '10.Shabi',
 'Mohamed',
 '11.Zakir',
 'Mugain',
 '12.Ferosh',
 'Khan',
 '13.Augustin',
 '14.Dr',
 '.',
 'T.C.Joseph',
 '..',
 'Petitioners',
 'Vs',
 '.',
 '1',
 '.',
 'Sheik',
 'Mydeen',
 '2',
 '.',
 'A.P.Nelson',
 '3',
 '.',
 'Chandrakala',
 'Ruben',
 '..',
 'Respondents',
 'Petition',
 'under',
 'Article',
 '227',
 'of',
 'the',
 'Constitution',
 'of',
 'India',
 'against',
 'the',
 'fair',
 'an

# Clean the sentences

In [47]:
clean_sentences = []

for sent in df['data'].to_list():
    clean_sentences.append(sentence_cleaner_v1(sent['text']))



In [48]:
final_df['clean_sent'] = clean_sentences
final_df.head()

,start,end,text,labels,tags,clean_sent
0,"[7, 73, 171, 237, 304, 350, 391, 425]","[40, 83, 182, 251, 311, 365, 400, 436]","[High Court Of Kerala At Ernakulam, T.R.Ajayan...","[[COURT], [PETITIONER], [RESPONDENT], [RESPOND...","[O, O, B-COURT, I-COURT, I-COURT, I-COURT, I-C...","[In, The, High, Court, Of, Kerala, At, Ernakul..."
1,"[21, 371, 445, 689, 741, 852, 1007, 1035, 1111...","[40, 383, 470, 698, 764, 869, 1031, 1047, 1121...","[Lokesh Kumar Sharma, Chanakyapuri, Jindal Ste...","[[JUDGE], [PETITIONER], [RESPONDENT], [RESPOND...","[O, B-RESPONDENT, O, O, O, B-JUDGE, I-JUDGE, E...","[In, The, Court, Of, Shri, Lokesh, Kumar, Shar..."
2,"[11, 98, 192, 208, 219, 237, 253, 265, 281, 29...","[45, 115, 204, 215, 233, 249, 261, 277, 295, 3...","[Madurai Bench Of Madras High Court, V.Ramasub...","[[COURT], [JUDGE], [PETITIONER], [PETITIONER],...","[O, O, B-COURT, I-COURT, I-COURT, I-COURT, I-C...","[Before, The, Madurai, Bench, Of, Madras, High..."
3,"[11, 99, 144, 305, 461, 593, 695, 1335, 1364, ...","[45, 112, 229, 330, 473, 604, 784, 1350, 1375,...","[Madurai Bench Of Madras High Court, T.Sudanth...","[[COURT], [JUDGE], [PETITIONER], [RESPONDENT],...","[O, O, B-COURT, I-COURT, I-COURT, I-COURT, I-C...","[Before, The, Madurai, Bench, Of, Madras, High..."
4,"[85, 435, 614, 773, 879, 997, 1276, 1598, 1658...","[119, 460, 637, 798, 887, 1035, 1296, 1617, 16...","[High Court Of Judicature At Bombay, Motiram M...","[[COURT], [PETITIONER], [RESPONDENT], [PETITIO...","[O, O, O, B-COURT, I-COURT, I-COURT, I-COURT, ...","[1, In, The, High, Court, Of, Judicature, At, ..."


## Word2idx

In [49]:
def get_dict_map(df , token_or_tag):
    """
    converts word to index

    [Need changes !!!]
    """
    tok2idx = {}
    idx2tok = {}

    vocab = []

    if token_or_tag == 'token':
        for sent in df['clean_sent'].to_list():
          vocab.extend( list(set(sent)) )

        vocab = set(vocab)

    else :
        for sent in df['tags'].to_list():
          vocab.extend( list(set(sent)) )
        
        vocab = set(vocab)


    idx2tok = {idx:tok for idx , tok in enumerate(vocab)}
    tok2idx = {tok:idx for idx , tok in enumerate(vocab)}

    return tok2idx, idx2tok



word2idx, idx2word = get_dict_map(final_df, 'token')
tag2idx, idx2tag = get_dict_map(final_df, 'tag')


In [50]:
"""
Padding and converting from word sequence to index sequence

"""

max_len = max([len(sent) for sent in final_df['clean_sent'].tolist()])
pad_idx = len(word2idx)
tag_pad_idx = tag2idx['O']
word_idx_sequence = []
tag_idx_sequence = []
pad = "PAD"

for sent in final_df['clean_sent'].to_list():
    #print(sent)
    word_idx_sequence.append([word2idx[word] for word in sent] + [pad_idx for _ in range(max_len - len(sent))])


for tag_seq in final_df['tags'].to_list():
    tag_idx_sequence.append([tag2idx[tag] for tag in tag_seq] + [tag_pad_idx for _ in range(max_len - len(tag_seq))])


In [51]:
final_df['word_idx'] = word_idx_sequence
final_df['tag_idx'] = tag_idx_sequence



In [52]:
len(tag2idx)

16

In [53]:
tag2idx

{'E-JUDGE': 0,
 'E-PETITIONER': 1,
 'B-LAWYER': 2,
 'E-RESPONDENT': 3,
 'I-RESPONDENT': 4,
 'O': 5,
 'I-PETITIONER': 6,
 'B-JUDGE': 7,
 'B-PETITIONER': 8,
 'E-LAWYER': 9,
 'I-JUDGE': 10,
 'I-LAWYER': 11,
 'E-COURT': 12,
 'B-COURT': 13,
 'B-RESPONDENT': 14,
 'I-COURT': 15}

In [55]:
import json

with open("tag2idx","w") as file:
  json.dump(tag2idx, file)

In [56]:
import json

with open("idx2tag","w") as file:
  json.dump(idx2tag, file)

In [54]:
word2idx

{'No.14/2017': 0,
 'P.R.': 1,
 '28.04.2014': 2,
 '478': 3,
 'K.Ramesh': 4,
 'S.N.Parashar': 5,
 'Y.Yadava': 6,
 'W.A.No.136/2006': 7,
 '4110/2011': 8,
 'No.414': 9,
 'W.P.No.11118/08': 10,
 'Ambar': 11,
 'Miglani': 12,
 '3199/1': 13,
 'Mahanagarpalika': 14,
 '054': 15,
 'Gosewade': 16,
 '11422': 17,
 'Jail': 18,
 'Prosecution': 19,
 'Labs': 20,
 'Dongrisab': 21,
 'Mallika': 22,
 'Rajbir': 23,
 'proprietor': 24,
 '5380': 25,
 'Sachar': 26,
 'Shankar': 27,
 'OSA.No.378': 28,
 'Banthia': 29,
 'Badharudeen': 30,
 'Jessee': 31,
 'I.A.No.589': 32,
 'at': 33,
 '506': 34,
 'Now': 35,
 'Enterprise': 36,
 'Giddappa': 37,
 'inside': 38,
 'Mukesh': 39,
 '20.9.2010': 40,
 'D.D.': 41,
 'Secretariat': 42,
 'class': 43,
 '4563/2008': 44,
 'Zeenath': 45,
 'Feng': 46,
 '4.M.Thavamani': 47,
 'Pavecha': 48,
 'Irvine': 49,
 'J.Usha': 50,
 'Khb': 51,
 '7428': 52,
 'Abdulsab': 53,
 '4.Mr': 54,
 '1983': 55,
 'M.C.O.P.No.264': 56,
 'left': 57,
 'written': 58,
 'Cou': 59,
 'Act': 60,
 'Mane': 61,
 '17876': 62,


In [57]:
import json

with open("word2idx","w") as file:
  json.dump(word2idx, file)

In [58]:
import json

with open("idx2word","w") as file:
  json.dump(idx2word, file)

In [59]:
final_df.to_csv("Final_DataFrame_Train_Preamble")